In [1]:
#バージョン確認
import sys
sys.version

'3.5.1 |Anaconda 4.0.0 (64-bit)| (default, Feb 16 2016, 09:49:46) [MSC v.1900 64 bit (AMD64)]'

<h2>ログインが必要なサイト（ニコニコ動画、大学のHP、限定的なデータベースなど）にログインするために必要なこと</h2>

ログインをしなければ見ることが出来ないコンテンツを含むサイトへのリクエストは単純にやっては上手く行かない。<br/>
例えば、ニコニコ動画はログインしなければ動画を見ることが出来ない。このようなサイトのスクレイピングを行う場合、<strong>セッション</strong>と<strong>クッキー</strong>という概念を理解してスクリプトを書くことが必要になる。

ウェブサーバーは、クライアントから何らかのリクエスト（GETやPOSTなど）を受け取り、適切なデータ(htmlなど)を返すという機能（レスポンス）を持つ。どんなユーザーであっても、同じデータを返すようなサイトであれば、どういうクライアントからのリクエストなのかを考慮する必要がない。<br/>
もし、ログイン機能を実装する場合、サーバーがリクエストを受け取ったとき、そのリクエストがログイン済みのユーザーからものなのか、それとも、ログインしていないユーザーからのものかを判別し、それぞれに別のデータを送らなければならない。

その際に有用なツールが<strong>クッキー</strong>である。クッキーは、サーバーがクライアントへ返すレスポンスに付加することができ、それを受け取ったクライアントがブラウザで保存することを前提にしている。そして、再びクライアントがリクエストを送るとき、そのクッキーを一緒に送ることで、サーバーがそのクライアントを識別できる。クッキーの中身は"name=value"の組み合わせである。<br/>
ログインした際、サーバーはクライアントに対し、何らかの方法で生成した<strong>セッションID</strong>というクッキーを送る。このセッションIDをリクエストに付加しているユーザーを同一とみなして、適切なページを送信する。このようなサーバーとクライアントの通信の保持が<strong>セッション</strong>である。ブラウザを閉じたりすると、そのクッキーが削除されてしまい、ログインした状態を保てなくなることもある。<br/>

つまり、ログインが必要なサイトに対しては、以下のような処理が必要になる。
<ul>
<li>ログインIDとパスワードをサーバーへ（主にPOSTリクエストで）送信する</li>
<li>ログインが成功すると、サーバーからセッションに関するクッキーが送信されるので、それらを保存する</li>
<li>次のリクエストからは、そのクッキーも一緒に送る</li>
</ul>

このような処理は、ブラウザでは全て自動でやってくれるが、スクリプトでリクエストを送る際は、全て自動でやらなければならない。<br/>
とは言うものの、Pythonにはこのようなクッキー関連の処理を自動化してくれる機能があるので、ありがたく使わせてもらう。<br/>
もちろん、自動化せずに、クッキーを自分で読み取り、自分でリクエスト（のヘッダー）に書くことも出来る。

<h6>ログインに送信すべき情報はどうやって調べるか</h6>

IDとパスワードを入力する画面のソースを見ると、多くのサイトのログイン画面では、inputタグの要素に入力された情報を、formタグのactionに対してPOSTで送信するということが書かれている。<br/>
ブラウザで開けるhtmlのサイトであれば、この処理はほとんど共通であるので、これを攻略できると、応用範囲が非常に広い。<br/>
POSTリクエストでは、クッキーとは別に、クライアント側からサーバーへデータを送ることが出来る。GETリクエスト（アドレスバーに打ち込んだときに送られて、アドレス後半の？以降がデータであるもの）とは違ってアドレスバーにデータが晒されることがないので、セキュリティなどを考えると、POSTリクエストになるのは自然な流れである。

POSTリクエストで送る内容は"name=value"というクッキーと同じ構文であるが、POSTリクエストでは、動画や音楽といった大容量のデータも送ることが出来る点でクッキーとは異なる。<br/>

具体的に、ログイン時にPOSTリクエストで送るべきデータは、formタグ内のinputタグのnameを列挙することで分かる。

このnameに対して、入力すべきデータを送信すればよい。例えば、全てのinputタグが２つで、それぞれのnameがidとpasswordの場合、
<ul>
<li>id=your_id</li>
<li>password=your_pw</li>
</ul>
と言った感じである。

最終的に、POSTリクエストは、formタグのactionに書かれているところに送ればいい。

In [2]:
#クッキーの入れ物のクラス
from http.cookiejar import CookieJar
cj = CookieJar()
#まだ中は空
cj

<CookieJar[]>

In [3]:
#クッキーを扱うためのrequest内のハンドラ（送信の際にURL等を処理したり、受信データを処理したり出来る的なもの）クラス。クッキージャーを入れる。
#送信のときに保存されたクッキーを一緒に送り、レスポンスとしてクッキーが届くと、それをクッキージャーに食べさせる機能を持つ
from urllib.request import HTTPCookieProcessor
http_cookie_processor = HTTPCookieProcessor(cj)

In [4]:
#URLを開くときに、引数として与えられたハンドラを連鎖的に（この場合一つだが）作用させられるもの。これを使ってURLを開く。この場合GETリクエスト。
from urllib.request import build_opener
opener = build_opener(http_cookie_processor)

試しにGoogleのトップページを開いてみると、ログインしているわけでもないのにクッキーが送られてくる。

In [5]:
res = opener.open("https://www.google.co.jp/")
body = res.read() #レスポンスを変数に保存。
cj

<CookieJar[Cookie(version=0, name='NID', value='92=Vl9OlKMSxTq2TqwBgylleM6DyeXeoNM7OnO0yboph1_TwA5ZpmXrzz_TbOMMmEJQhh5HdmtDaIoaNYwxqxHZ7OukVmvnky2O-mnJl2M8pBvrllLhe5cCecvlYbJvVbFM', port=None, port_specified=False, domain='.google.co.jp', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=1497983319, discard=False, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)]>

ちなみに、レスポンスのHTMLを表示すると以下のようになる。CSSや画像がないところを見ると、HTMLには基本的な構造しか含まれていないことが分かる。

In [6]:
from IPython.core.display import display, HTML
display(HTML(body.decode("shift-jis")))

,,検索オプション言語ツール


<h6>基本的なクッキーの使い方</h6>

CookieJarの中には、ドメインごとにクッキーをまとめたCookieオブジェクトが入っており、それらはイテレータで取り出すことが出来る。<br/>
したがって、forループで回したり、リストに変換することが出来る。

In [7]:
for cookie in cj:
    print(cookie)
list(cj)

<Cookie NID=92=Vl9OlKMSxTq2TqwBgylleM6DyeXeoNM7OnO0yboph1_TwA5ZpmXrzz_TbOMMmEJQhh5HdmtDaIoaNYwxqxHZ7OukVmvnky2O-mnJl2M8pBvrllLhe5cCecvlYbJvVbFM for .google.co.jp/>


[Cookie(version=0, name='NID', value='92=Vl9OlKMSxTq2TqwBgylleM6DyeXeoNM7OnO0yboph1_TwA5ZpmXrzz_TbOMMmEJQhh5HdmtDaIoaNYwxqxHZ7OukVmvnky2O-mnJl2M8pBvrllLhe5cCecvlYbJvVbFM', port=None, port_specified=False, domain='.google.co.jp', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=1497983319, discard=False, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)]

クッキーを消去することも出来る。

In [8]:
cj.clear()
cj

<CookieJar[]>

複数のドメインからクッキーが送られてくると、複数のクッキーが保存される。

In [9]:
opener.open("http://www.yahoo.co.jp/")
opener.open("https://www.google.co.jp/")
opener.open("http://stackoverflow.com/")
cj

<CookieJar[Cookie(version=0, name='NID', value='92=h_MhO1Auj9X8dU5FqMt5aHIYzS3H2qfVX4oeGugY-CMF42G2YE7Ies0oXwm11r6lGQj3RoIaojGiUqhRpAYJga8_JsgjNlXfhLaoo3ORgobsa3v-g6O1pYlbEv6aFSBX', port=None, port_specified=False, domain='.google.co.jp', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=1497983319, discard=False, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False), Cookie(version=0, name='prov', value='412fcd58-fbd4-bb11-5951-3b1e91206284', port=None, port_specified=False, domain='.stackoverflow.com', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=2682374400, discard=False, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)]>

In [10]:
#一つ目のクッキー
cookie = list(cj)[0]
cookie

Cookie(version=0, name='NID', value='92=h_MhO1Auj9X8dU5FqMt5aHIYzS3H2qfVX4oeGugY-CMF42G2YE7Ies0oXwm11r6lGQj3RoIaojGiUqhRpAYJga8_JsgjNlXfhLaoo3ORgobsa3v-g6O1pYlbEv6aFSBX', port=None, port_specified=False, domain='.google.co.jp', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=1497983319, discard=False, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)

In [11]:
#クッキーの値の取り出し
cookie.value

'92=h_MhO1Auj9X8dU5FqMt5aHIYzS3H2qfVX4oeGugY-CMF42G2YE7Ies0oXwm11r6lGQj3RoIaojGiUqhRpAYJga8_JsgjNlXfhLaoo3ORgobsa3v-g6O1pYlbEv6aFSBX'